In [37]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [38]:
data = pd.read_csv('../Data/Shakespeare_data.csv')

In [39]:
data.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [40]:
strategy = tf.distribute.MirroredStrategy()
#Nutzt die GPU

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Nachzulesen hier: https://www.tensorflow.org/api_docs/python/tf/distribute/MirroredStrategy

# Manipulation der Daten
- Wir brauchen nur die Spalte *"PlayerLine"*
- Am besten machen wir alles Lower oder Upper Case

## Diskussion
- Ersetzten von I mit 1, II mit 2 usw.?
    - Hat das einen Effekt auf den output oder ist das Irrelevant? 

In [41]:
training_data = data['PlayerLine'].str.lower()
training_data.head()

0                                                act i
1                         scene i. london. the palace.
2    enter king henry, lord john of lancaster, the ...
3               so shaken as we are, so wan with care,
4           find we a time for frighted peace to pant,
Name: PlayerLine, dtype: object

# Tokenizer

In [42]:
#Tokenizer erstellen und fitten
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['PlayerLine'])

#Wörter in Integer umwandeln
sequences = tokenizer.texts_to_sequences(data['PlayerLine'])

#Maximale länge bestimmen (Wichtig für die Layers!!!)
max_sequence_length = max(len(seq) for seq in sequences)

#Mit hilfe von 0 alle sequenzen auf die selbe lenge bringen
sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

vocab_size = len(tokenizer.word_index) + 1

print(sequences[:3])

[[ 315    3    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]
 [ 135    3  802    1  571    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0  

In [43]:
total_words_sum = len(tokenizer.word_index)+1
print(total_words_sum)

25576


In [44]:
max_seq_len = max(len(x) for x in sequences)
print(max_seq_len)

167


# Modell

In [45]:
#TODO: Testen, strucktur mit LSTM überdenken
from keras.src.layers import Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU

def generator_model():
    model = Sequential()
    model.add(Embedding(total_words_sum, 100, input_length = max_seq_len-1)),
    model.add(Bidirectional(LSTM(64, return_sequences = True))),
    model.add(Bidirectional(LSTM(32))),
    model.add(Dense(64, activation = 'relu')),
    model.add(Dense(total_words_sum, activation = 'softmax'))
    return model

# Discriminator

In [46]:
#TODO

# Training

In [47]:
#TODO